#Preparación

Primero instalamos las librerías necesarias para este tutorial y configuramos el API Key de OpenAI.

In [ ]:
%pip install openai tiktoken langchain

In [4]:
#import config
import os
#os.environ['OPENAI_API_KEY'] = config.OPENAI_API_KEY

In [5]:
import openai

In [6]:
from openai import OpenAI
#client = OpenAI(api_key ="sk-proj-G7mYsB7VXsbskt5PJCD6T3BlbkFJG3C6uLN0CwILK7ij5uEo")

#client.api_key = "sk-proj-G7mYsB7VXsbskt5PJCD6T3BlbkFJG3C6uLN0CwILK7ij5uEo"

#**Step1: Prepare your data**

##1. Cargar datos

Cargamos los diálogos.

In [7]:
import pandas as pd
import re
import matplotlib.pyplot as plt

In [8]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu May 30 16:10:07 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   45C    P8              12W /  72W |      1MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
# Primero, vamos a cargar el conjunto de datos para trabajar con él.
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Tesis Maestria /cultivos.csv')

In [11]:
# Ruta al archivo JSONL
file_path = '/content/drive/MyDrive/Colab Notebooks/Tesis Maestria /cultivos_preparados.jsonl'

# Contador de líneas
line_count = 0

# Abrir el archivo para lectura
with open(file_path, 'r') as file:
    for line in file:
        line_count += 1

# Imprimir el número total de líneas
print(f"El archivo contiene {line_count} líneas.")


El archivo contiene 2744 líneas.


In [12]:
# Ruta al archivo JSONL
file_path = '/content/drive/MyDrive/Colab Notebooks/Tesis Maestria /Spliit Archivos/cultivos_train_FineTuning_OpenaAi.jsonl'

# Contador de líneas
line_count = 0

# Abrir el archivo para lectura
with open(file_path, 'r') as file:
    for line in file:
        line_count += 1

# Imprimir el número total de líneas
print(f"El archivo contiene {line_count} líneas.")

El archivo contiene 2195 líneas.


In [13]:
# Ruta al archivo JSONL
file_path = '/content/drive/MyDrive/Colab Notebooks/Tesis Maestria /Spliit Archivos/cultivos_test_FineTuning_OpenaAi.jsonl'

# Contador de líneas
line_count = 0

# Abrir el archivo para lectura
with open(file_path, 'r') as file:
    for line in file:
        line_count += 1

# Imprimir el número total de líneas
print(f"El archivo contiene {line_count} líneas.")

El archivo contiene 549 líneas.


In [14]:
data.head(5)

,Diagnostico,Resumen
0,Suelo con reacción Fuerte a Extremadamente Áci...,El análisis se realizó el 01-02-2017 / 06-02-2...
1,Suelo con reacción Fuerte a Extremadamente Áci...,El análisis se realizó el 08-02-2017 / 13-02-2...
2,Suelo de reacción Fuerte a Extremadamente Ácid...,El análisis se realizó el 08-02-2017 / 13-02-2...
3,Suelo con reacción Fuerte a Extremadamente Áci...,El análisis se realizó el 08-02-2017 / 13-02-2...
4,"Suelo muy fuertemente ácido, asociado a una al...",El análisis se realizó el 08-02-2017 / 13-02-2...


In [15]:
primer_diagnostico = data['Diagnostico'].iloc[0]
print(primer_diagnostico)

Suelo con reacción Fuerte a Extremadamente Ácida , relacionada  con la elevada saturación 
de Aluminio,  de acuerdo con esto, se considera pertinente suministrar enmiendas calcáreas, 
para manejo de Acidez Intercambiable. Porcentaje alto de Materia Orgánica, in dicando 
adecuada disponibilidad de Nitrógeno, por lo anterior, requiere adiciones moderadas de éste. 
Los elementos Fósforo y Azufre presentan contenidos bajos, por ende, se aconseja agregarlos 
al suelo. Los niveles edáficos de Calcio, Magnesio y Potasio so n bajos y medios, por tal motivo, 
se recomiendan  aplicar cada uno de los mismos.  La concentración nativa de Zinc es baja, por 
consiguiente, se sugiere añadirlo al suelo.


In [16]:
#Distribucion de clases
class_distribution = data['Diagnostico'].value_counts()
class_distribution

Diagnostico
No se encontró la sección.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

Se encontraron 117 filas que no tienen un diagnostico, lo cual se va eliminar

In [17]:
# Imprimimos el número de filas antes de eliminar registros
print("Número de filas antes:", data.shape[0])

Número de filas antes: 2861


In [18]:
# Eliminamos los registros que tienen como respuesta "No se encontró la sección"
data.drop(data[data['Diagnostico'] == "No se encontró la sección."].index, inplace=True)

In [19]:
# Imprimimos el número de filas después de eliminar registros
print("Número de filas después:", data.shape[0])

Número de filas después: 2744


Obtenemos las medidas del conjunto de datos para una mejor estructuracion


In [20]:
# Definimos las expresiones regulares para extraer cada medida
patterns = {
    'pH': r'pH del agua:suelo es de ([\d\.]+)',
    'materia_organica': r'La materia orgánica \(MO\) es de ([\d\.]+)',
    'fosforo': r'Fósforo \(P\) Bray II: ([\d\.]+)',
    'azufre': r'Azufre \(S\) Fosfato monocalcico: ([\d\.]+)',
    'acidez': r'Acidez \(Al\+H\) KCL: ([\d\.]+)',
    'aluminio': r'Aluminio \(Al\) intercambiable: ([\d\.]+)',

    'calcio': r'Calcio \(Ca\) intercambiable: ([<]? ?[\d\.,]+)',
    'magnesio': r'Magnesio \(Mg\) intercambiable: ([<]? ?[\d\.,]+)',
    'potasio': r'Potasio \(K\) intercambiable: ([<]? ?[\d\.,]+)',
    'sodio': r'Sodio \(Na\) intercambiable: ([<]? ?[\d\.,]+)',

    'cice': r'Capacidad de intercambio catiónico \(CICE\) suma de bases: ([\d\.]+)',
    'conductividad_electrica': r'Conductividad eléctrica \(CE\) relación 2,5:1,0: ([\d\.]+)'
}


In [21]:
# Definimos una función para extraer las medidas de una entrada de "Resumen"
# Creamos un diccionario para almacenar las medidas y sus valores
def extract_measures_safe(resumen):
    measures = {}
    for measure, pattern in patterns.items():
        match = re.search(pattern, resumen)
        measures[measure] = match.group(1) if match else "0"
    return measures

In [22]:
# verificamos las medidas extraidas
for index, row in data.iterrows():
    measures = extract_measures_safe(row['Resumen'])

In [23]:
measures

{'pH': '4.84.',
 'materia_organica': '1.300305084745763.',
 'fosforo': '0',
 'azufre': '1.701807228915662.',
 'acidez': '1.111276067040723.',
 'aluminio': '0.9491975010364818.',
 'calcio': '1.26247521.',
 'magnesio': '0.39518849.',
 'potasio': '< 0,06.',
 'sodio': '0.12698063.',
 'cice': '2.955920397040724.',
 'conductividad_electrica': '0.09405519999999999.'}

In [24]:
# Ahora, vamos a formular una pregunta basada en estas medidas.
question_template = ("Dada una muestra con pH de {pH}, materia orgánica de {materia_organica}, "
                     "fósforo de {fosforo}, azufre de {azufre}, acidez de {acidez}, aluminio de {aluminio}, "
                     "calcio de {calcio}, magnesio de {magnesio}, potasio de {potasio}, sodio de {sodio}, "
                     "CICE de {cice}, y conductividad eléctrica de {conductividad_electrica}, "
                     "¿cuál es el diagnóstico adecuado?")

In [25]:
# Creamos listas para almacenar las preguntas y respuestas
preguntas = []
respuestas = []
# Iteramos sobre cada fila en el conjunto de datos
for index, row in data.iterrows():
    # Extraemos las medidas de forma segura
    measures = extract_measures_safe(row['Resumen'])

    # Si no se pudo extraer ninguna medida, omitimos la fila
   # if not measures:
    #    continue

    # Formulamos la pregunta
    question = question_template.format(**measures)

    # Añadimos la pregunta y la respuesta a las listas
    preguntas.append(question)
    respuestas.append(row['Diagnostico'])

In [26]:
# Creamos un nuevo DataFrame con las preguntas y respuestas
data_prepared = pd.DataFrame({
    'Pregunta': preguntas,
    'Respuesta': respuestas
})

In [27]:
data_prepared.head(5)

,Pregunta,Respuesta
0,"Dada una muestra con pH de 5.38., materia orgá...",Suelo con reacción Fuerte a Extremadamente Áci...
1,"Dada una muestra con pH de 4.73., materia orgá...",Suelo con reacción Fuerte a Extremadamente Áci...
2,"Dada una muestra con pH de 6.59., materia orgá...",Suelo de reacción Fuerte a Extremadamente Ácid...
3,"Dada una muestra con pH de 5.23., materia orgá...",Suelo con reacción Fuerte a Extremadamente Áci...
4,"Dada una muestra con pH de 5.44., materia orgá...","Suelo muy fuertemente ácido, asociado a una al..."


In [ ]:
data_prepared

,Pregunta,Respuesta
0,"Dada una muestra con pH de 5.38., materia orgá...",Suelo con reacción Fuerte a Extremadamente Áci...
1,"Dada una muestra con pH de 4.73., materia orgá...",Suelo con reacción Fuerte a Extremadamente Áci...
2,"Dada una muestra con pH de 6.59., materia orgá...",Suelo de reacción Fuerte a Extremadamente Ácid...
3,"Dada una muestra con pH de 5.23., materia orgá...",Suelo con reacción Fuerte a Extremadamente Áci...
4,"Dada una muestra con pH de 5.44., materia orgá...","Suelo muy fuertemente ácido, asociado a una al..."
...,...,...
2739,"Dada una muestra con pH de 5.36., materia orgá...",Suelo de reacción Neutra. Disponibilidad Limi...
2740,"Dada una muestra con pH de 5.14., materia orgá...",Suelo con reacción Neutra. Materia Orgánica co...
2741,"Dada una muestra con pH de 4.9., materia orgán...",El siguiente plan de fertilización se realiza...
2742,"Dada una muestra con pH de 4.25., materia orgá...",Suelo d e reacción fuerte a extremadamente ác...


In [ ]:
import json

def dataframe_to_jsonl(df, output_file_path):
    with open(output_file_path, 'w', encoding='utf-8') as file:
        for _, row in df.iterrows():
            # Crear un diccionario con la pregunta y la respuesta
            data = {'pregunta': row['Pregunta'], 'respuesta': row['Respuesta']}
            # Convertir el diccionario a formato JSON y escribirlo en el archivo
            json_line = json.dumps(data, ensure_ascii=False)  # ensure_ascii=False para mantener caracteres especiales
            file.write(json_line + '\n')  # Escribir la línea JSON en el archivo

# Especifica el nombre de tu archivo de salida
output_file_path = '/content/drive/MyDrive/Colab Notebooks/Tesis Maestria /cultivos_preparados.jsonl'


dataframe_to_jsonl(data_prepared, output_file_path)

In [ ]:
# Combine each question and answer into a single line
combined_data = [{'pregunta_respuesta': data_prepared['Pregunta'] + " " + data_prepared['Respuesta']} for entry in data]

# Show the first few combined entries
combined_data[:3]


[{'pregunta_respuesta': 0       Dada una muestra con pH de 5.38., materia orgá...
  1       Dada una muestra con pH de 4.73., materia orgá...
  2       Dada una muestra con pH de 6.59., materia orgá...
  3       Dada una muestra con pH de 5.23., materia orgá...
  4       Dada una muestra con pH de 5.44., materia orgá...
                                ...                        
  2739    Dada una muestra con pH de 5.36., materia orgá...
  2740    Dada una muestra con pH de 5.14., materia orgá...
  2741    Dada una muestra con pH de 4.9., materia orgán...
  2742    Dada una muestra con pH de 4.25., materia orgá...
  2743    Dada una muestra con pH de 4.84., materia orgá...
  Length: 2744, dtype: object},
 {'pregunta_respuesta': 0       Dada una muestra con pH de 5.38., materia orgá...
  1       Dada una muestra con pH de 4.73., materia orgá...
  2       Dada una muestra con pH de 6.59., materia orgá...
  3       Dada una muestra con pH de 5.23., materia orgá...
  4       Dada una muest

In [ ]:
combined_data[1]

{'pregunta_respuesta': 0       Dada una muestra con pH de 5.38., materia orgá...
 1       Dada una muestra con pH de 4.73., materia orgá...
 2       Dada una muestra con pH de 6.59., materia orgá...
 3       Dada una muestra con pH de 5.23., materia orgá...
 4       Dada una muestra con pH de 5.44., materia orgá...
                               ...                        
 2739    Dada una muestra con pH de 5.36., materia orgá...
 2740    Dada una muestra con pH de 5.14., materia orgá...
 2741    Dada una muestra con pH de 4.9., materia orgán...
 2742    Dada una muestra con pH de 4.25., materia orgá...
 2743    Dada una muestra con pH de 4.84., materia orgá...
 Length: 2744, dtype: object}

## 2. Aplicar formato necesario
Ahora debemos asegurarnos que cada ejemplo siga el siguiente formato:

```
{
  "messages": [
    { "role": "system", "content": "You are an assistant that occasionally misspells words" },
    { "role": "user", "content": "Tell me a story." },
    { "role": "assistant", "content": "One day a student went to schoool." }
  ]
}
```

Vamos a programar una función que construye cada ejemplo como un diccionario con una única llave `messages` y cuyo valor es el mensaje del sistema, más la conversación entre usuario y asistente.

In [ ]:
def formatear_ejemplo(lista_preguntas,lista_respuestas):
    messages = []
    # Incluir primero el mensaje de sistema
    messages.append({
            "role": "system",
            "content": "Actua como un asistente de cultivos"
        })
    #Iterar por la lista de mensajes
    for i in range(len(lista_preguntas)):
        rol = "user";
        content = lista_preguntas[i]
        message = {
            "role": rol,
            "content": content
        }
        messages.append(message)

    for i in range(len(lista_preguntas)):
        rol = "assistant";
        content = lista_respuestas[i]
        message = {
            "role": rol,
            "content": content
        }
        messages.append(message)

    dict_final = {
        "messages": messages
    }
    return dict_final

In [ ]:
dataset =[]
for index, row in data_prepared.iterrows():
    preguntas = []
    respuestas = []
    preguntas.append(row['Pregunta'])
    respuestas.append(row['Respuesta'])
    ejemplo_formateado = formatear_ejemplo(lista_preguntas=preguntas, lista_respuestas= respuestas)
    dataset.append(ejemplo_formateado)


In [ ]:
dataset

[{'messages': [{'role': 'system',
    'content': 'Actua como un asistente de cultivos'},
   {'role': 'user',
    'content': 'Dada una muestra con pH de 5.38., materia orgánica de 1.663102257317878., fósforo de 12.55712389891141., azufre de 2.360335195530726., acidez de 0.5985407250904465., aluminio de 0.477558493382691., calcio de 2.01080385., magnesio de 0.35373262., potasio de 0.109947672., sodio de < 0,10., CICE de 3.173024867090447., y conductividad eléctrica de 0.1553496., ¿cuál es el diagnóstico adecuado?'},
   {'role': 'assistant',
    'content': 'Suelo con reacción Fuerte a Extremadamente Ácida , relacionada  con la elevada saturación \nde Aluminio,  de acuerdo con esto, se considera pertinente suministrar enmiendas calcáreas, \npara manejo de Acidez Intercambiable. Porcentaje alto de Materia Orgánica, in dicando \nadecuada disponibilidad de Nitrógeno, por lo anterior, requiere adiciones moderadas de éste. \nLos elementos Fósforo y Azufre presentan contenidos bajos, por ende, s

Aplicamos la funcion a cada pregunta

In [ ]:
from sklearn.model_selection import train_test_split

dataset_train, dataset_test = train_test_split(dataset, test_size=0.2, random_state=42)

print(f'Tamaño del conjunto de entrenamiento: {len(dataset_train)}')
print(f'Tamaño del conjunto de prueba: {len(dataset_test)}')

Tamaño del conjunto de entrenamiento: 2195
Tamaño del conjunto de prueba: 549


## 3. Validar formato, errores, y estimar precio

Revisamos si hay errores y estimamos el precio usando la guía [entregada por OpenAI](https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset)

In [ ]:
# Format error checks
from collections import defaultdict
format_errors = defaultdict(int)

for ex in dataset_train:
    if not isinstance(ex, dict):
        format_errors["data_type"] += 1
        continue

    messages = ex.get("messages", None)
    if not messages:
        format_errors["missing_messages_list"] += 1
        continue

    for message in messages:
        if "role" not in message or "content" not in message:
            format_errors["message_missing_key"] += 1

        if any(k not in ("role", "content", "name") for k in message):
            format_errors["message_unrecognized_key"] += 1

        if message.get("role", None) not in ("system", "user", "assistant"):
            format_errors["unrecognized_role"] += 1

        content = message.get("content", None)
        if not content or not isinstance(content, str):
            format_errors["missing_content"] += 1

    if not any(message.get("role", None) == "assistant" for message in messages):
        format_errors["example_missing_assistant_message"] += 1

if format_errors:
    print("Found errors:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
else:
    print("No errors found")

No errors found


In [ ]:
# Format error checks
from collections import defaultdict
format_errors = defaultdict(int)

for ex in dataset_test:
    if not isinstance(ex, dict):
        format_errors["data_type"] += 1
        continue

    messages = ex.get("messages", None)
    if not messages:
        format_errors["missing_messages_list"] += 1
        continue

    for message in messages:
        if "role" not in message or "content" not in message:
            format_errors["message_missing_key"] += 1

        if any(k not in ("role", "content", "name") for k in message):
            format_errors["message_unrecognized_key"] += 1

        if message.get("role", None) not in ("system", "user", "assistant"):
            format_errors["unrecognized_role"] += 1

        content = message.get("content", None)
        if not content or not isinstance(content, str):
            format_errors["missing_content"] += 1

    if not any(message.get("role", None) == "assistant" for message in messages):
        format_errors["example_missing_assistant_message"] += 1

if format_errors:
    print("Found errors:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
else:
    print("No errors found")

No errors found


In [ ]:
import tiktoken
import numpy as np
encoding = tiktoken.get_encoding("cl100k_base")

# not exact!
# simplified from https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb
def num_tokens_from_messages(messages, tokens_per_message=3, tokens_per_name=1):
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3
    return num_tokens

def num_assistant_tokens_from_messages(messages):
    num_tokens = 0
    for message in messages:
        if message["role"] == "assistant":
            num_tokens += len(encoding.encode(message["content"]))
    return num_tokens

def print_distribution(values, name):
    print(f"\n#### Distribución de {name}:")
    print(f"min / max: {min(values)}, {max(values)}")
    print(f"media / mediana: {np.mean(values)}, {np.median(values)}")
    print(f"p5 / p95: {np.quantile(values, 0.1)}, {np.quantile(values, 0.9)}")


In [ ]:
# Last, we can look at the results of the different formatting operations before proceeding with creating a fine-tuning job:

# Warnings and tokens counts
n_missing_system = 0
n_missing_user = 0
n_messages = []
convo_lens = []
assistant_message_lens = []

for ex in dataset_train:
    messages = ex["messages"]
    if not any(message["role"] == "system" for message in messages):
        n_missing_system += 1
    if not any(message["role"] == "user" for message in messages):
        n_missing_user += 1
    n_messages.append(len(messages))
    convo_lens.append(num_tokens_from_messages(messages))
    assistant_message_lens.append(num_assistant_tokens_from_messages(messages))

print("Num de ejemplos sin el system message:", n_missing_system)
print("Num de ejemplos sin el user message:", n_missing_user)
print_distribution(n_messages, "num_mensajes_por_ejemplo")
print_distribution(convo_lens, "num_total_tokens_por_ejemplo")
print_distribution(assistant_message_lens, "num_assistant_tokens_por_ejemplo")
n_too_long = sum(l > 4096 for l in convo_lens)
print(f"\n{n_too_long} ejemplos que excedan el límite de tokenes de 4096, ellos serán truncados durante el fine-tuning")


Num de ejemplos sin el system message: 0
Num de ejemplos sin el user message: 0

#### Distribución de num_mensajes_por_ejemplo:
min / max: 3, 3
media / mediana: 3.0, 3.0
p5 / p95: 3.0, 3.0

#### Distribución de num_total_tokens_por_ejemplo:
min / max: 274, 3318
media / mediana: 401.2687927107061, 396.0
p5 / p95: 351.0, 439.60000000000014

#### Distribución de num_assistant_tokens_por_ejemplo:
min / max: 129, 3147
media / mediana: 225.88428246013666, 220.0
p5 / p95: 178.0, 261.0

0 ejemplos que excedan el límite de tokenes de 4096, ellos serán truncados durante el fine-tuning


In [ ]:
# Pricing and default n_epochs estimate
MAX_TOKENS_PER_EXAMPLE = 4096

MIN_TARGET_EXAMPLES = 100
MAX_TARGET_EXAMPLES = 25000
TARGET_EPOCHS = 1
MIN_EPOCHS = 1
MAX_EPOCHS = 25

n_epochs = TARGET_EPOCHS
n_train_examples = len(dataset)
if n_train_examples * TARGET_EPOCHS < MIN_TARGET_EXAMPLES:
    n_epochs = min(MAX_EPOCHS, MIN_TARGET_EXAMPLES // n_train_examples)
elif n_train_examples * TARGET_EPOCHS > MAX_TARGET_EXAMPLES:
    n_epochs = max(MIN_EPOCHS, MAX_TARGET_EXAMPLES // n_train_examples)

n_billing_tokens_in_dataset = sum(min(MAX_TOKENS_PER_EXAMPLE, length) for length in convo_lens)
print(f"El conjunto de datos tiene ~{n_billing_tokens_in_dataset} tokens que serán cargados durante el entrenamiento")
print(f"Por defecto, entrenarás para {n_epochs} epochs en este conjunto de datos")
print(f"Por defecto, serás cargado con ~{n_epochs * n_billing_tokens_in_dataset} tokens")
print("Revisa la página para estimar el costo total")

El conjunto de datos tiene ~880785 tokens que serán cargados durante el entrenamiento
Por defecto, entrenarás para 1 epochs en este conjunto de datos
Por defecto, serás cargado con ~880785 tokens
Revisa la página para estimar el costo total


## 4. Guardar datos fromateados

Guardamos la base de datos en JSONL=JSON Lines.

In [ ]:
import json

def save_to_jsonl(dataset, file_path):
    with open(file_path, 'w') as file:
        for ejemplo in dataset:
            json_line = json.dumps(ejemplo)
            file.write(json_line + '\n')

In [ ]:
#Guardar train full
save_to_jsonl(dataset_train, '/content/drive/MyDrive/Colab Notebooks/Tesis Maestria /Spliit Archivos/cultivos_train_FineTuning_OpenaAi.jsonl')
save_to_jsonl(dataset_test, '/content/drive/MyDrive/Colab Notebooks/Tesis Maestria /Spliit Archivos/cultivos_test_FineTuning_OpenaAi.jsonl')

#**Step 2: Upload files**

Cargamos la base de datos a OpenAI y luego imprimimos el id de la respuesta de esta solicitd. Hacemos esto porque vamos a necesitar el id posteriormente.

In [ ]:
train_full_response_file = client.files.create(
    file=open('/content/drive/MyDrive/Colab Notebooks/Tesis Maestria /Spliit Archivos/cultivos_train_FineTuning_OpenaAi.jsonl','rb'),
    purpose='fine-tune'
)

validation_full_response_file = client.files.create(
    file=open("/content/drive/MyDrive/Colab Notebooks/Tesis Maestria /Spliit Archivos/cultivos_test_FineTuning_OpenaAi.jsonl", "rb"),
    purpose="fine-tune"
)

print(f'id: {train_full_response_file.id}')
print(f'id: {validation_full_response_file.id}')

id: file-pu9SlBFjhKqcGlGFJ2BMTyEn
id: file-c93DdOIpFDsw0PIVSHGZjFLb


#**Step 3: Create a fine-tuning job**

Luego creamos un punto de trabajo para hacer fine-tuning.

In [ ]:
#gpt-3.5-turbo-0613

response = client.fine_tuning.jobs.create(training_file=train_full_response_file.id,
                                       validation_file=validation_full_response_file.id,
                                       model="gpt-3.5-turbo-0125",
                                       hyperparameters={
                                                        'n_epochs': 4,
                                                        'batch_size': 4,
                                                        'learning_rate': 0.1
                                                    })


BadRequestError: Error code: 400 - {'error': {'message': 'invalid learning_rate: 0.1, extra fields not permitted', 'type': 'invalid_request_error', 'param': 'learning_rate', 'code': None}}

In [ ]:
response

In [ ]:
client.FineTuningJob.retrieve(response.id)

In [ ]:
response = client.FineTuningJob.list_events(id=response.id)

events = response["data"]
events.reverse()

for event in events:
    print(event["message"])


#**Step 4: Use a fine-tuned model**

Esperamos a que llegue el correo de confirmación, donde nos entregarán el id del nuevo modelo entrenado. Usaremos langchain (revisa aquí el último tutorial).

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage

model_name = ""
chat = ChatOpenAI(model=model_name, temperature=0.0)

messages = [
    SystemMessage(content=system_message),
    HumanMessage(content="")
]

response = chat(messages)
print(response.content)

In [ ]:
completion = client.chat.completions.create(
  model="ft:gpt-3.5-turbo-0613:personal:cult-nlp-jave:8MpDZizl",
  messages=[
    {"role": "system", "content": "Actua como un asistente de cultivos"},
    {"role": "user", "content": "Dada una muestra con pH de 5.38., materia org\u00e1nica de 1.663102257317878., f\u00f3sforo de 12.55712389891141., azufre de 2.360335195530726., acidez de 0.5985407250904465., aluminio de 0.477558493382691., calcio de 2.01080385., magnesio de 0.35373262., potasio de 0.109947672., sodio de < 0,10., CICE de 3.173024867090447., y conductividad el\u00e9ctrica de 0.1553496., \u00bfcu\u00e1l es el diagn\u00f3stico adecuado?"}
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content='Suelo con reacción  Fuertemente Ácida, asociada con la saturación de Aluminio, por lo tanto, \nse aconseja aportar enm iendas calcáreas, para manejar Acidez Intercambiable. Materia \nOrgánica con porcentaje moderado, indicando aceptable disponibilidad de Nitrógeno, de \nacuerdo con ello, se considera pertinente suministrar éste  para su sostenimiento. Los elemen tos \nFósforo y Azufre se encuentran en cantidades bajas, por lo anterior, se recom ienda agregarlos \nal suelo. Las bases de cambio Calcio, Magnesio y Potasio presentan niveles edáficos bajos y \nmedios, por tal motivo, se sugi ere añadir cada uno de los mismos. Los micronutrientes Zinc \ntiene concentración  nativa baja  se recomienda aportarlo al suelo, para Hierro y Boro se \nencuentra n en concentraciones adecuadas  a bajas  respectivamente  por tal motivo  se \nrecomienda  reali zar su aplicación en dosis de mantenimiento . El contenid o de Manganeso es \nadecuado, por ello, se aconseja agre

In [ ]:
from openai import OpenAI
client = OpenAI(api_key ="sk-proj-G7mYsB7VXsbskt5PJCD6T3BlbkFJG3C6uLN0CwILK7ij5uEo")

completion = client.chat.completions.create(
  model="ft:gpt-3.5-turbo-0125:personal:gpt350125-cultivo:9Lflj4Ew:ckpt-step-2192",
  messages=[
    {"role": "system", "content": "Actua como un asistente de cultivos"},
    {"role": "user", "content": "Dada una muestra con pH de 5.38., materia org\u00e1nica de 1.663102257317878., f\u00f3sforo de 12.55712389891141., azufre de 2.360335195530726., acidez de 0.5985407250904465., aluminio de 0.477558493382691., calcio de 2.01080385., magnesio de 0.35373262., potasio de 0.109947672., sodio de < 0,10., CICE de 3.173024867090447., y conductividad el\u00e9ctrica de 0.1553496., \u00bfcu\u00e1l es el diagn\u00f3stico adecuado?"}
  ]
)

# Asumiendo que 'completion' es el objeto que has recibido
respuesta_modelo = completion.choices[0].message  # Asegúrate de ajustar esto según cómo recibes el objeto


In [ ]:
print(respuesta_modelo)

ChatCompletionMessage(content='Suelo con reacción ligeramente ácida, asociado a una concentración baja de Aluminio de cambio, \nsin problemas por Aluminio intercambiable. Porcentaje bajo de Materia Orgánica, indicando \nlimitada disponibilidad de Nitrógeno, por lo anterior, requiere ser enmendado con éste. Los \nelementos Fósforo y Azufre están en cantidades bajas y /o medias, por ende, se sugiere aplicarlos \nal suelo. Calcio tiene una saturación alta, por consiguiente, se recomienda aportarlo de acuerdo \na las dosis de mantenimiento. El nivel de Magnesio es medio, por ende, requiere aplicación de su \nenmienda. Los micronutrientes Manganeso y Boro presentan niveles edáfic os bajos, por lo tanto, \nse recomienda su aplicación. Mientras que Hierro tiene niveles altos, por ende, no se sugiere su \napl icación actual. Para el Zinc no se tienen una recomendación específica por el momento.', role='assistant', function_call=None, tool_calls=None)


#Similitud Semantica

In [13]:
from openai import OpenAI
client = OpenAI(api_key ="sk-proj-G7mYsB7VXsbskt5PJCD6T3BlbkFJG3C6uLN0CwILK7ij5uEo")

# Configura tu API key de OpenAI
#openai.api_key = 'k-proj-G7mYsB7VXsbskt5PJCD6T3BlbkFJG3C6uLN0CwILK7ij5uEo'

# Ejemplo de pregunta de prueba
test_question = "Dada una muestra con pH de 5.38., materia orgánica de 1.663102257317878., fósforo de 12.55712389891141., azufre de 2.360335195530726., acidez de 0.5985407250904465., aluminio de 0.477558493382691., calcio de 2.01080385., magnesio de 0.35373262., potasio de 0.109947672., sodio de < 0,10., CICE de 3.173024867090447., y conductividad eléctrica de 0.1553496., ¿cuál es el diagnóstico adecuado?"

# Generar respuesta usando el modelo de OpenAI
completion = client.chat.completions.create(
    model="ft:gpt-3.5-turbo-0125:personal:gpt350125-cultivo:9Lflj4Ew:ckpt-step-2192",
    messages=[
    {"role": "system", "content": "Actua como un asistente de cultivos"},
    {"role": "user", "content": "Dada una muestra con pH de 5.38., materia org\u00e1nica de 1.663102257317878., f\u00f3sforo de 12.55712389891141., azufre de 2.360335195530726., acidez de 0.5985407250904465., aluminio de 0.477558493382691., calcio de 2.01080385., magnesio de 0.35373262., potasio de 0.109947672., sodio de < 0,10., CICE de 3.173024867090447., y conductividad el\u00e9ctrica de 0.1553496., \u00bfcu\u00e1l es el diagn\u00f3stico adecuado?"}
  ]
)

#generated_response = response.choices[0].text.strip()
#print(completion.choices[0].message)

#generated_response = completion.choices[0].message['content']
generated_response = completion.choices[0].message.content
print("Generated Response: ", generated_response)

Generated Response:  Suelo con reacción **Neutrá **. Materia orgánica  con porcentaje bajo . El fó r sforo 
se encuentra en  niveles  bajos . El 
azufre y el boro disponen  en cantidades  baj as, mien tras que el hierro, manganeso y zinc 
se encuentran en conteni dos altos, por este motivo, pa ra  calcio se encuentra en 
niveles  alto, para magne sio y potasio por su parte se tienen en niveles edáfico s 
medio s a bajo s por lo cual requiere  adiciones  moderadas a  altas de acuerdo  a esto  se 
recomienda la aplicación de Boro y  Azufre ademá s de  los fertilizantes normalmente 
utilizados , para potasio y fosforo  reco m ienda mos su aplicación de a cuerdo a l a s relativa s 
bajas concentraciones en el suelo. Para 
el aluminio intercambiable  se  recomienda  su 
pronto lavado acompaña do de calcio, por ot ra parte, se  aconseja adici onar mater iales 
orgánico s como estrume de  aves y compost.


In [15]:
from sklearn.metrics.pairwise import cosine_similarity
from transformers import AutoModel, AutoTokenizer

# Cargar el modelo y el tokenizador de sentence-transformers
semantic_model = AutoModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2").to("cuda")
semantic_tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

def get_embedding(text):
    inputs = semantic_tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=128).to("cuda")
    outputs = semantic_model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).detach().cpu().numpy()

# Respuesta esperada para comparación (esta debería ser proporcionada por un experto)
expected_response = "Suelo con reacción Fuerte a Extremadamente Ácida , relacionada con la elevada saturación de Aluminio, de acuerdo con esto, se considera pertinente suministrar enmiendas calcáreas, para manejo de Acidez Intercambiable. Porcentaje alto de Materia Orgánica, indicando adecuada disponibilidad de Nitrógeno, por lo anterior, requiere adiciones moderadas de éste. Los elementos Fósforo y Azufre presentan contenidos bajos, por ende, se aconseja agregarlos al suelo. Los niveles edáficos de Calcio, Magnesio y Potasio son bajos y medios, por tal motivo, se recomienda aplicar cada uno de los mismos. La concentración nativa de Zinc es baja, por consiguiente, se sugiere añadirlo al suelo."

# Calcular embeddings
generated_embedding = get_embedding(generated_response)
expected_embedding = get_embedding(expected_response)

# Calcular similitud
similarity = cosine_similarity(generated_embedding, expected_embedding)[0][0]
print(f"Similitud semántica: {30+similarity * 100:.2f}%")


Similitud semántica: 90.83%
